In [1]:
import os
!wget https://github.com/RUCAIBox/POPE/blob/main/POPEv2/dataset/annotations.json -P /vlm/annotations/

--2026-01-17 11:39:05--  https://github.com/RUCAIBox/POPE/blob/main/POPEv2/dataset/annotations.json
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/vlm/annotations/annotations.json’

annotations.json        [ <=>                ] 713.89K  --.-KB/s    in 0.1s    

2026-01-17 11:39:05 (5.50 MB/s) - ‘/vlm/annotations/annotations.json’ saved [731028]



In [ ]:
# from pycocotools.coco import COCO

# coco = COCO("/kaggle/input/coco-2014-dataset-for-yolov3/coco2014/annotations/instances_train2014.json")

# img_id = 123
# ann_ids = coco.getAnnIds(imgIds=img_id)
# anns = coco.loadAnns(ann_ids)

# objects = set()
# for ann in anns:
#     cat = coco.loadCats(ann["category_id"])[0]["name"]
#     objects.add(cat)

In [1]:
import json
import os
from tqdm import tqdm

LLAVA_JSON = "llava_instruct_150k.json"

COCO_DIRS = [
    "coco2014/images/train2014",
    "coco2014/images/val2014",
    "coco2014/images/test2014",
]

OUTPUT_JSONL = "My Drive/llava_train.jsonl"



In [8]:
def find_img(img_name):
    coco_fix = [
        "COCO_train2014_",
        "COCO_val2014",
        "COCO_tes2014",
    ]

    for coco_dir in COCO_DIRS:
        for prefix in coco_fix:
            path = os.path.join(coco_dir, prefix + img_name)
            if os.path.exists(path):
                return path
    return None

In [5]:
# with open(LLAVA_JSON, "r") as f:
#     llava_data = json.load(f)

# valid = 0
# missing = 0

# with open(OUTPUT_JSONL, "w") as out:
#     for sample in tqdm(llava_data):
#         image_name = sample.get("image", "")
#         image_path = find_img(image_name)

#         if image_path is None:
#             missing += 1
#             continue

#         record = {
#             "id": sample.get("id"),
#             "image": image_path,
#             "conversations": sample["conversations"]
#         }

#         out.write(json.dumps(record) + "\n")
#         valid += 1

# print(f"Valid samples: {valid}")
# print(f"Missing images: {missing}")
# print(f"Saved to: {OUTPUT_JSONL}")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

ValueError: mount failed

In [9]:
import random

with open(OUTPUT_JSONL) as f:
    lines = f.readlines()

sample = json.loads(random.choice(lines))
print(sample["image"])
print(sample["conversations"][0])


FileNotFoundError: [Errno 2] No such file or directory: 'My Drive/llava_train.jsonl'

In [ ]:
# !pip install --upgrade transformers accelerate bitsandbytes sentencepiece protobuf pillow torch

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    CLIPVisionModel, CLIPImageProcessor,
    LlamaForCausalLM, LlamaTokenizer,
    get_linear_schedule_with_warmup
)
from PIL import Image
import json
from pathlib import Path
from tqdm import tqdm
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Cell 3: LLaVA Model Architecture
class LLaVAModel(nn.Module):
    def __init__(
        self,
        vision_model_name="openai/clip-vit-large-patch14",
        llm_model_name="meta-llama/Llama-2-7b-chat-hf",  # or use smaller model
        projection_dim=4096
    ):
        super().__init__()
        
        # Vision encoder (CLIP)
        self.vision_tower = CLIPVisionModel.from_pretrained(vision_model_name)
        self.vision_processor = CLIPImageProcessor.from_pretrained(vision_model_name)
        
        # Freeze vision encoder
        for param in self.vision_tower.parameters():
            param.requires_grad = False
        
        # Get vision feature dimension
        vision_hidden_size = self.vision_tower.config.hidden_size  # 1024 for CLIP-L
        
        # Projection layer to map vision features to LLM dimension
        self.mm_projector = nn.Sequential(
            nn.Linear(vision_hidden_size, projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )
        
        # Language model
        self.llm = LlamaForCausalLM.from_pretrained(
            llm_model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            load_in_8bit=True  # Use 8-bit quantization to save memory
        )
        
        # Tokenizer
        self.tokenizer = LlamaTokenizer.from_pretrained(llm_model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"
        
        # Special token for image
        self.image_token_id = self.tokenizer.convert_tokens_to_ids("<image>")
        
    def encode_images(self, images):
        """Encode images using CLIP vision encoder"""
        with torch.no_grad():
            image_features = self.vision_tower(images).last_hidden_state
        # Project to LLM dimension
        image_features = self.mm_projector(image_features)
        return image_features
    
    def forward(self, input_ids, attention_mask, images=None, labels=None):
        """
        Forward pass
        input_ids: tokenized text with <image> placeholder
        images: preprocessed images
        labels: target tokens for training
        """
        if images is not None:
            # Encode images
            image_features = self.encode_images(images)  # [B, 257, 4096]
            
            # Find positions of <image> tokens
            batch_size = input_ids.shape[0]
            new_input_embeds = []
            new_labels = [] if labels is not None else None
            
            for i in range(batch_size):
                # Get text embeddings
                text_embeds = self.llm.model.embed_tokens(input_ids[i])
                
                # Find image token position
                image_token_positions = (input_ids[i] == self.image_token_id).nonzero(as_tuple=True)[0]
                
                if len(image_token_positions) > 0:
                    # Replace <image> token with actual image features
                    img_pos = image_token_positions[0]
                    
                    # Concatenate: text before image + image features + text after image
                    new_embed = torch.cat([
                        text_embeds[:img_pos],
                        image_features[i],
                        text_embeds[img_pos+1:]
                    ], dim=0)
                    
                    new_input_embeds.append(new_embed)
                    
                    if labels is not None:
                        # Adjust labels for image features
                        new_label = torch.cat([
                            labels[i][:img_pos],
                            torch.full((image_features.shape[1],), -100, device=labels.device, dtype=labels.dtype),
                            labels[i][img_pos+1:]
                        ], dim=0)
                        new_labels.append(new_label)
                else:
                    new_input_embeds.append(text_embeds)
                    if labels is not None:
                        new_labels.append(labels[i])
            
            # Pad sequences to same length
            max_len = max(x.shape[0] for x in new_input_embeds)
            
            padded_embeds = []
            padded_labels = [] if labels is not None else None
            new_attention_mask = []
            
            for i in range(batch_size):
                embed = new_input_embeds[i]
                pad_len = max_len - embed.shape[0]
                
                # Pad embeddings
                padded_embed = torch.cat([
                    embed,
                    torch.zeros(pad_len, embed.shape[1], device=embed.device, dtype=embed.dtype)
                ], dim=0)
                padded_embeds.append(padded_embed)
                
                # Pad attention mask
                attn_mask = torch.cat([
                    torch.ones(embed.shape[0], device=device),
                    torch.zeros(pad_len, device=device)
                ], dim=0)
                new_attention_mask.append(attn_mask)
                
                # Pad labels
                if labels is not None:
                    label = new_labels[i]
                    padded_label = torch.cat([
                        label,
                        torch.full((pad_len,), -100, device=label.device, dtype=label.dtype)
                    ], dim=0)
                    padded_labels.append(padded_label)
            
            input_embeds = torch.stack(padded_embeds)
            attention_mask = torch.stack(new_attention_mask)
            labels = torch.stack(padded_labels) if labels is not None else None
            
            # Forward through LLM with embeddings
            outputs = self.llm(
                inputs_embeds=input_embeds,
                attention_mask=attention_mask,
                labels=labels,
                return_dict=True
            )
        else:
            # Text-only forward
            outputs = self.llm(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels,
                return_dict=True
            )
        
        return outputs

print("Model architecture defined!")

Model architecture defined!


In [ ]:
# Cell 4: Dataset Class
class LLaVADataset(Dataset):
    def __init__(self, jsonl_path, vision_processor, tokenizer, max_length=512):
        self.data = []
        with open(jsonl_path, 'r') as f:
            for line in f:
                self.data.append(json.loads(line))
        
        self.vision_processor = vision_processor
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        # Add image token if not exists
        if "<image>" not in self.tokenizer.get_vocab():
            self.tokenizer.add_tokens(["<image>"])
        
        print(f"Loaded {len(self.data)} samples")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        # Load and process image
        image_path = item['image']
        try:
            image = Image.open(image_path).convert('RGB')
            pixel_values = self.vision_processor(images=image, return_tensors="pt")['pixel_values'][0]
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            # Return dummy image
            pixel_values = torch.zeros(3, 224, 224)
        
        # Format conversation
        conversations = item['conversations']
        
        # Build conversation text
        conversation_text = ""
        for conv in conversations:
            if conv['from'] == 'human':
                # Replace <image> placeholder with special token
                text = conv['value'].replace('<image>', '<image>')
                conversation_text += f"USER: {text}\n"
            else:  # gpt
                conversation_text += f"ASSISTANT: {conv['value']}\n"
        
        # Tokenize
        # For training, we want input_ids to be the full conversation
        # and labels to mask out the user parts (only train on assistant responses)
        full_text = conversation_text
        
        # Tokenize the full conversation
        encoded = self.tokenizer(
            full_text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        input_ids = encoded['input_ids'][0]
        attention_mask = encoded['attention_mask'][0]
        
        # Create labels (mask USER parts, only train on ASSISTANT parts)
        labels = input_ids.clone()
        
        # Find ASSISTANT token positions
        assistant_token = self.tokenizer.encode("ASSISTANT:", add_special_tokens=False)
        user_token = self.tokenizer.encode("USER:", add_special_tokens=False)
        
        # Simple approach: mask everything except what comes after "ASSISTANT:"
        # More sophisticated version would track exact positions
        labels_list = labels.tolist()
        
        # Mask all positions initially
        for i in range(len(labels_list)):
            labels_list[i] = -100
        
        # Find and unmask assistant responses
        text_ids = input_ids.tolist()
        i = 0
        while i < len(text_ids):
            # Look for "ASSISTANT:" pattern
            if i + len(assistant_token) <= len(text_ids):
                if text_ids[i:i+len(assistant_token)] == assistant_token:
                    # Found ASSISTANT, unmask from here until next USER or end
                    j = i + len(assistant_token)
                    while j < len(text_ids):
                        # Check if we hit USER token
                        if j + len(user_token) <= len(text_ids):
                            if text_ids[j:j+len(user_token)] == user_token:
                                break
                        # Check if we hit padding
                        if text_ids[j] == self.tokenizer.pad_token_id:
                            break
                        labels_list[j] = text_ids[j]
                        j += 1
                    i = j
                else:
                    i += 1
            else:
                i += 1
        
        labels = torch.tensor(labels_list)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'pixel_values': pixel_values,
            'labels': labels
        }

print("Dataset class defined!")

Dataset class defined!


In [ ]:
# Cell 5: Initialize Model and Dataset
print("Initializing model...")

# Use TinyLlama - it's open and doesn't require authentication
model = LLaVAModel(
    vision_model_name="openai/clip-vit-large-patch14",
    llm_model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    projection_dim=2048
)

train_dataset = LLaVADataset(
    jsonl_path = "./llava_train.jsonl",
    vision_processor=model.vision_processor,
    tokenizer=model.tokenizer,
    max_length=512
)

# Resize token embeddings if we added new tokens
model.llm.resize_token_embeddings(len(model.tokenizer))
model = model.to(device)

# Cell 6: Training Configuration
BATCH_SIZE = 2  # Small batch size for memory constraints
GRAD_ACCUMULATION_STEPS = 8  # Effective batch size = 2 * 8 = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
WARMUP_STEPS = 100
SAVE_STEPS = 500

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

# Only train projection layer and optionally LoRA for LLM
trainable_params = list(model.mm_projector.parameters())

# Option: Add LoRA to LLM for parameter-efficient fine-tuning
# This would require peft library - uncomment if you want to use it
# from peft import LoraConfig, get_peft_model
# lora_config = LoraConfig(
#     r=8,
#     lora_alpha=16,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )
# model.llm = get_peft_model(model.llm, lora_config)
# trainable_params.extend(model.llm.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=LEARNING_RATE)

total_steps = len(train_loader) * EPOCHS // GRAD_ACCUMULATION_STEPS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {WARMUP_STEPS}")

# Cell 7: Training Loop
def train_epoch(model, dataloader, optimizer, scheduler, epoch, grad_accum_steps):
    model.train()
    # Keep vision tower in eval mode
    model.vision_tower.eval()
    
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    
    for step, batch in enumerate(progress_bar):
        # Move to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            images=pixel_values,
            labels=labels
        )
        
        loss = outputs.loss / grad_accum_steps
        loss.backward()
        
        total_loss += loss.item()
        
        # Gradient accumulation
        if (step + 1) % grad_accum_steps == 0:
            torch.nn.utils.clip_grad_norm_(trainable_params, max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # Update progress bar
        progress_bar.set_postfix({
            'loss': loss.item() * grad_accum_steps,
            'lr': scheduler.get_last_lr()[0]
        })
        
        # Save checkpoint
        if (step + 1) % SAVE_STEPS == 0:
            checkpoint_path = f"./llava_checkpoint_epoch{epoch+1}_step{step+1}.pt"
            torch.save({
                'epoch': epoch,
                'step': step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': total_loss / (step + 1),
            }, checkpoint_path)
            print(f"\nCheckpoint saved to {checkpoint_path}")
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Cell 8: Run Training
print("Starting training...")

for epoch in range(EPOCHS):
    avg_loss = train_epoch(
        model,
        train_loader,
        optimizer,
        scheduler,
        epoch,
        GRAD_ACCUMULATION_STEPS
    )
    
    print(f"\nEpoch {epoch+1}/{EPOCHS} - Average Loss: {avg_loss:.4f}")
    
    # Save epoch checkpoint
    checkpoint_path = f"./llava_epoch{epoch+1}_final.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': avg_loss,
    }, checkpoint_path)
    print(f"Epoch checkpoint saved to {checkpoint_path}")

print("\nTraining complete!")

# Save final model
final_model_path = "model/llava_final_model.pt"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to {final_model_path}")

Initializing model...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loaded 157712 samples


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Total training steps: 29571
Warmup steps: 100
Starting training...


Epoch 1:   0%|          | 0/78856 [00:00<?, ?it/s]c:\Users\DzungTa\anaconda3\envs\vlm\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
# !pip install -U bitsandbyte